In [1]:
%cd ../

/Users/amyrfrancisco/Developer/projects/tm-rag-amyr


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("meta-llama/llama-4-scout-17b-16e-instruct", model_provider="groq")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/Users/amyrfrancisco/Developer/projects/tm-rag-amyr/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("docs", glob="**/*.pdf", show_progress=True, use_multithreading=True)
docs = loader.load()

len(docs)

100%|██████████| 1/1 [00:11<00:00, 11.05s/it]


1

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=250)
all_splits = text_splitter.split_documents(docs)

len(all_splits)

387

In [ ]:
from langchain_chroma import Chroma

# vector_store = Chroma(
#     embedding_function=embeddings,
#     persist_directory="chroma_langchain_db"
# )

vector_store = Chroma.from_documents(all_splits, embeddings)

In [ ]:
# _ = vector_store.add_documents(documents=all_splits)

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Refer to the 'context' as the 'document' but don't mention 'According to the document' or the likes when answering questions. Provide the answer directly.
Question: {question} 
Context: {context} 
Answer:
"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict, Annotated

class Search(TypedDict):
    query: Annotated[str, ..., "Search query to run"]
    topic: Annotated[str, ..., "Topic of the query"]

class State(TypedDict):
    question: str
    query: Search
    context: List[Document]
    answer:str

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import MessagesState

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=4)
    serialized = "\n\n----------\n\n".join(
        (f"Source: {doc.metadata["source"]}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

tools = ToolNode([retrieve])

def generate(state: MessagesState):
    """Generate answer."""
    recent_tool_messages = []
    # Get the most recent tool messages
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    # Reverse the list to preserve chronological order
    tool_messages = recent_tool_messages[::-1]

    # You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    # Refer to the 'context' as the 'document' but don't mention 'According to the document' or the likes when answering questions. Provide the answer directly.
    # Question: {question} 
    # Context: {context} 
    # Answer:

    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "Your name is ChatPDF, you are created by Amyr. "
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    response = llm.invoke(prompt)
    return {"messages": [response]}

In [ ]:
from langgraph.graph import StateGraph
from langgraph.graph import END
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver

graph_builder = StateGraph(MessagesState)

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"}
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "amyr"}}

In [ ]:
# from langchain_core.messages import HumanMessage

# query = "anw, so far what did i ask?"
# input_message = [HumanMessage(query)]

# for step in graph.stream(
#     {"messages": input_message},
#     stream_mode="values",
#     config=config,
# ):
#     step["messages"][-1].pretty_print()

from langchain_core.messages import HumanMessage

query = "tell me about our revenue on 2023"
input_message = [HumanMessage(query)]

response = graph.invoke({"messages": input_message}, config)

response["messages"][-1].pretty_print()

In [ ]:
last_message = response["messages"][-1]
latest_tool_message = last_message if last_message.type == "tool" else None

if latest_tool_message:
    print(latest_tool_message.content)